In [47]:
import pandas as pd
import numpy as np


In [48]:
# Load the CSV file
file_path = '../data/Actual Generation per Production Type_DK1.csv'
df = pd.read_csv(file_path)

# Convert 'n/e' to 0 and numeric columns
numerical_columns = df.columns[2:]
for col in numerical_columns:
    df[col] = pd.to_numeric(df[col].replace('n/e', 0), errors='coerce').fillna(0)

# Sum all numerical columns
df['Total Generation [MW]'] = df[numerical_columns].sum(axis=1)

# Display the result
df[['MTU', 'Total Generation [MW]']]


C:\Users\Thorn\AppData\Local\Temp\ipykernel_4032\1199330051.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = pd.to_numeric(df[col].replace('n/e', 0), errors='coerce').fillna(0)


,MTU,Total Generation [MW]
0,01.01.2025 00:00 - 01.01.2025 01:00 (CET/CEST),4988.0
1,01.01.2025 01:00 - 01.01.2025 02:00 (CET/CEST),4830.0
2,01.01.2025 02:00 - 01.01.2025 03:00 (CET/CEST),4497.0
3,01.01.2025 03:00 - 01.01.2025 04:00 (CET/CEST),4410.0
4,01.01.2025 04:00 - 01.01.2025 05:00 (CET/CEST),4474.0
...,...,...
8756,31.12.2025 19:00 - 31.12.2025 20:00 (CET/CEST),0.0
8757,31.12.2025 20:00 - 31.12.2025 21:00 (CET/CEST),0.0
8758,31.12.2025 21:00 - 31.12.2025 22:00 (CET/CEST),0.0
8759,31.12.2025 22:00 - 31.12.2025 23:00 (CET/CEST),0.0


In [49]:
# Load the imbalance data
imbalance_file_path = '../data/Imbalance_year_DK1.csv'
imbalance_df = pd.read_csv(imbalance_file_path)

# Convert 'n/e' to 0 in the imbalance data and make it numeric
imbalance_df['Total Imbalance [MWh] - IBA|DK1'] = pd.to_numeric(
    imbalance_df['Total Imbalance [MWh] - IBA|DK1'].replace('n/e', 0), errors='coerce'
).fillna(0)

# Adjust sign based on "Deficit" or "Surplus"
imbalance_df['Total Imbalance [MWh] - IBA|DK1'] = imbalance_df.apply(
    lambda row: -row['Total Imbalance [MWh] - IBA|DK1'] if row['Situation'] == 'Deficit' else row['Total Imbalance [MWh] - IBA|DK1'],
    axis=1
)

# Display the result
imbalance_df[['Imbalance settlement period CET/CEST', 'Total Imbalance [MWh] - IBA|DK1']]


,Imbalance settlement period CET/CEST,Total Imbalance [MWh] - IBA|DK1
0,01.01.2025 00:00 - 01.01.2025 01:00,-226.0
1,01.01.2025 01:00 - 01.01.2025 02:00,-282.0
2,01.01.2025 02:00 - 01.01.2025 03:00,-351.0
3,01.01.2025 03:00 - 01.01.2025 04:00,-298.0
4,01.01.2025 04:00 - 01.01.2025 05:00,-322.0
...,...,...
8756,31.12.2025 19:00 - 31.12.2025 20:00,0.0
8757,31.12.2025 20:00 - 31.12.2025 21:00,0.0
8758,31.12.2025 21:00 - 31.12.2025 22:00,0.0
8759,31.12.2025 22:00 - 31.12.2025 23:00,0.0


In [50]:
for i in range(len(df)):
    df.loc[i, 'Total Generation [MW]'] += imbalance_df.loc[i, 'Total Imbalance [MWh] - IBA|DK1']

# Display the updated result
df[['MTU', 'Total Generation [MW]']]

,MTU,Total Generation [MW]
0,01.01.2025 00:00 - 01.01.2025 01:00 (CET/CEST),4762.0
1,01.01.2025 01:00 - 01.01.2025 02:00 (CET/CEST),4548.0
2,01.01.2025 02:00 - 01.01.2025 03:00 (CET/CEST),4146.0
3,01.01.2025 03:00 - 01.01.2025 04:00 (CET/CEST),4112.0
4,01.01.2025 04:00 - 01.01.2025 05:00 (CET/CEST),4152.0
...,...,...
8756,31.12.2025 19:00 - 31.12.2025 20:00 (CET/CEST),0.0
8757,31.12.2025 20:00 - 31.12.2025 21:00 (CET/CEST),0.0
8758,31.12.2025 21:00 - 31.12.2025 22:00 (CET/CEST),0.0
8759,31.12.2025 22:00 - 31.12.2025 23:00 (CET/CEST),0.0


In [51]:
# Load the dmi data
dmi_file_path = '../data/DMI_data.csv'
dmi_df = pd.read_csv(dmi_file_path)

# Convert 'Datetime' to a proper datetime format for filtering
dmi_df['Datetime'] = pd.to_datetime(dmi_df['Datetime'])

# Filter only the data from 2025 onwards
dmi_df = dmi_df[dmi_df['Datetime'] >= "2025-01-01"]

# Display the first few rows after filtering
dmi_df.head()


,Datetime,mean_temp,mean_wind_speed
8814,2025-01-01 00:00:00+00:00,7.7,11.1
8815,2025-01-01 01:00:00+00:00,7.8,10.8
8816,2025-01-01 02:00:00+00:00,7.9,10.7
8817,2025-01-01 03:00:00+00:00,7.9,10.4
8818,2025-01-01 04:00:00+00:00,7.9,10.3


In [52]:
# Convert 'MTU' (Time Slot) in Total Generation data to a datetime format
df['MTU'] = df['MTU'].apply(lambda x: pd.to_datetime(x.split(" - ")[0], format="%d.%m.%Y %H:%M"))

# Remove timezone info from DMI timestamps
dmi_df['Datetime'] = dmi_df['Datetime'].dt.tz_localize(None)

In [53]:
# Rename 'Datetime' column in DMI data to match for easy merging
dmi_df.rename(columns={'Datetime': 'Time Slot'}, inplace=True)
df.rename(columns={'MTU': 'Time Slot'}, inplace=True)

# Display the first few rows of both datasets to confirm changes
df[['Time Slot', 'Total Generation [MW]']].head(), dmi_df.head()
len(df), len(dmi_df)

(8761, 1186)

In [54]:
seq_length = 24
total_generation = df['Total Generation [MW]'].values
mean_temp = dmi_df['mean_temp'].values
mean_wind_speed = dmi_df['mean_wind_speed'].values
imbalance = imbalance_df['Total Imbalance [MWh] - IBA|DK1'].values

min_length = min(len(total_generation), len(mean_temp), len(mean_wind_speed), len(imbalance))

total_generation = total_generation[:min_length]
mean_temp = mean_temp[:min_length]
mean_wind_speed = mean_wind_speed[:min_length]
imbalance = imbalance[:min_length]


In [55]:
# Prepare the input (X) and target (Y) sequences
X = []
Y = []

for i in range(len(total_generation) - seq_length - 1):  # Ensure we don't go out of bounds
    # Input: [Total Generation, Mean Temp, Mean Wind Speed] for 24 hours
    X_seq = np.stack([
        total_generation[i:i+seq_length],
        mean_temp[i:i+seq_length],
        mean_wind_speed[i:i+seq_length]
    ], axis=0)  # Shape (3, 24)

    # Target: Imbalance for the next 24 hours
    Y_seq = imbalance[i+1:i+1+seq_length]  # Shape (24,)

    X.append(X_seq)
    Y.append(Y_seq)

# Convert to numpy arrays
X = np.array(X)  # Shape: (num_samples, 3, 24)
Y = np.array(Y)  # Shape: (num_samples, 24)

# Display the shape of the processed data
X.shape, Y.shape

((1161, 3, 24), (1161, 24))